In [13]:
from sympy import symbols, simplify, Not, And, Or, Implies, satisfiable
from itertools import combinations
from functools import reduce
#debugging
import pdb
%pdb on

def shannon_expansion(f, s):

    print("5) entro in shannon_expansion(f,s)")
    result = (s & f.subs(s, True)) | (~s & f.subs(s, False))
    print("shannon", result)
    print("shannon result", result.subs)
    return result.subs({True: True, False: False})

def common(sublist, group):

    print("3) entro in common(sublist, group)")
    if len(sublist) == 0:
        return group
    list_atoms_sets = [s.atoms() - {True, False} for s in group]
    print("list_atoms_sets", list_atoms_sets)
    list_atoms = reduce(lambda s1, s2: s1.union(s2), list_atoms_sets)
    print("list_atoms", list_atoms)
    added = False
    print("sublist", sublist)
    for f in sublist:
        if len(list_atoms.intersection(f.atoms())) > 0:
            group.append(f)
            sublist.remove(f)
            added = True
    if not added:
        return group
    return common(sublist, group)

def common_subs(form):

    print("2) entro in common_subs(form)")
    subs = list(form.args)
    all_groups = []
    while len(subs) > 0:
        sub1 = subs[0]
        group = [sub1]
        subs.remove(sub1)
        group = common(subs, group)
        all_groups.append(group)
    return all_groups

def common_atoms(subformulas):

    print("4) entro in common_atoms(subformulas)")
    subformulas_atoms = [set(filter(lambda a: a not in [True, False], e.atoms())) for e in subformulas]
    print("subformulas_atoms" ,subformulas_atoms)
    common_atoms_set = set()
    for i, sub1 in enumerate(subformulas_atoms):
        for sub2 in subformulas_atoms[i + 1:]:
            common_atoms_set = common_atoms_set.union(sub1.intersection(sub2))
    return common_atoms_set

def NNF2DNNF(nnf):

    print("1) entro in NNF2DNNF(nnf)")
    atoms = list(filter(lambda x: x not in [True, False], nnf.atoms()))
    print("atoms", atoms)
    if len(atoms) <= 1:

        print()
        return nnf.subs({True | False: True, False | True: True, True & False: False, False & True: False})
    all_groups = common_subs(nnf)
    print("all_groups", all_groups)
    result_subformulas = []
    for subformulas in all_groups:
        if len(subformulas) == 1:
            result_subformulas.append(NNF2DNNF(subformulas[0]))
        else:
            args = list(filter(lambda x: x != True, subformulas))
            new_expr = reduce(lambda e1, e2: e1 & e2, args)
            cmn_atoms = common_atoms(subformulas)
            common_atom = cmn_atoms.pop()
            result_subformulas.append(NNF2DNNF(shannon_expansion(new_expr, common_atom)))
    return reduce(lambda e1, e2: e1 & e2, result_subformulas).subs({True | False: True, False | True: True, True & False: False, False & True: False})

def DNNF2dDNNF(dnnf):

    print("entro in DNNF2dDNNF(dnnf)")
    pdb.set_trace()
    atoms = list(filter(lambda x: x not in [True, False], dnnf.atoms()))

    print("atoms DNNF2dDNNF", atoms)
    if dnnf.func == And:
        return reduce(lambda x, y: x & y, deterministic_subs).subs({True | False: True, False | True: True, True & False: False, False & True: False})
    result = reduce(lambda x, y: x | y, deterministic_subs).subs({True | False: True, False | True: True, True & False: False, False & True: False})
    for subexp1, subexp2 in combinations(deterministic_subs, 2):
        form = subexp1 & subexp2
        if satisfiable(form.subs({True: True, False: False})):
            sep = form.atoms().difference({True, False}).pop()
            expanded = shannon_expansion(result, sep).subs({True | False: True, False | True: True, True & False: False, False & True: False})
            return DNNF2dDNNF(expanded)
    return result

Automatic pdb calling has been turned ON


In [14]:
from sympy import symbols, Or, And, Not

def smooth_left(formula):

    print("entro in smooth_left(formula)")
    if isinstance(formula, Or):
        p_set = formula.args[1].free_symbols - formula.args[0].free_symbols
    return formula

def smooth_right(formula):

    print("entro in smooth_right(formula)")
    if isinstance(formula, Or):
        p_set = formula.args[0].free_symbols - formula.args[1].free_symbols
        if p_set:
            p = p_set.pop()
            return Or(formula.args[0], And(formula.args[1], Or(p, Not(p))))
    return formula

def get_subformulas(formula):

    print("entro in get_subformulas(formula)")
    subformulas = set()
    def traverse(node):
        subformulas.add(node)
        for arg in node.args:
            traverse(arg)
    traverse(formula)
    print("traverse ", traverse(formula))
    return list(subformulas)

def ddnnf2sdNNF(ddnnf_formula):

    print("entro in ddnnf2sdNNF(ddnnf_formula)")
    new_formula = ddnnf_formula
    while True:
        args = get_subformulas(new_formula)
        print("args", args)
        transformed = False
        for i in range(len(args)):
            if isinstance(args[i], Or):
                new_sub = smooth_right(args[i])
                new_sub = smooth_left(new_sub)
                if new_sub != args[i]:
                    new_formula = new_formula.subs(args[i+1], new_sub)
                    transformed = True
        if not transformed or new_formula == formula:
            break
        ddnnf_formula = new_formula
    return new_formula

In [15]:
def model_counting_sdnf(sdNNF_formula, weights):

    print("6) entro in model_counting_sdnf(sdNNF_formula, weights)")
    if isinstance(sdNNF_formula, Or):
        print("or")
        result = 0
        for arg in sdNNF_formula.args:
            result += model_counting_sdnf(arg, weights)
        return result
    elif isinstance(sdNNF_formula, And):
        print("and")
        result = 1
        for arg in sdNNF_formula.args:
            result *= model_counting_sdnf(arg, weights)
        return result
    else:
        print("else")
        if sdNNF_formula in weights:
            return weights[sdNNF_formula]

In [16]:
#test1

A, B, C = symbols('A B C')
print(A)
print(type(A))
print(B)
print(C)

formula = And(Or(A, B), Or(Not(A), C))  # (A ∨ B) ∧ (¬A ∨ C)

nnf = simplify(formula)
print("\n nnf , ", nnf)
dnnf = NNF2DNNF(nnf)
print("\n dnnf" , dnnf)

weights = {
    A: 0.7, Not(A): 0.3,
    B: 0.6, Not(B): 0.4,
    C: 0.8, Not(C): 0.2,
}

wmc = model_counting_sdnf(dnnf, weights)

print("\n Formula:", formula)
print("DNNF:", dnnf)
print("Weighted Model Count:", wmc)


A
<class 'sympy.core.symbol.Symbol'>
B
C

 nnf ,  (A & C) | (B & ~A)
1) entro in NNF2DNNF(nnf)
atoms [A, C, B]
2) entro in common_subs(form)
3) entro in common(sublist, group)
list_atoms_sets [{A, C}]
list_atoms {A, C}
sublist [B & ~A]
3) entro in common(sublist, group)
all_groups [[A & C, B & ~A]]
4) entro in common_atoms(subformulas)
subformulas_atoms [{A, C}, {A, B}]
5) entro in shannon_expansion(f,s)
shannon False
shannon result <bound method Basic.subs of False>
1) entro in NNF2DNNF(nnf)
atoms []


 dnnf False
6) entro in model_counting_sdnf(sdNNF_formula, weights)
else

 Formula: (A | B) & (C | ~A)
DNNF: False
Weighted Model Count: None


In [17]:
from sympy import symbols, And, Or, Not

# Step 1: Definizione dei letterali
a, b, c = symbols('{a b c}')

# Step 2: Formula di esempio
formula = Or(And(a, Not(b)), c)

# Step 3: Conversione in DNNF e poi in d-DNNF
dnnf = NNF2DNNF(formula)
ddnnf = DNNF2dDNNF(dnnf)
sddnnf = ddnnf2sdNNF(ddnnf)

# Step 4: Definizione dei pesi (es. uniformi)
weights = {a: 0.5, Not(a): 0.5, b: 0.5, Not(b): 0.5, c: 0.5, Not(c): 0.5}

# Step 5: Conteggio dei modelli
count = model_counting_sdnf(sddnnf, weights)
print(f"Model count: {count}")

1) entro in NNF2DNNF(nnf)
atoms [b, c}, {a]
2) entro in common_subs(form)
3) entro in common(sublist, group)
list_atoms_sets [{c}}]
list_atoms {c}}
sublist [{a & ~b]
3) entro in common(sublist, group)
all_groups [[c}], [{a & ~b]]
1) entro in NNF2DNNF(nnf)
atoms [c}]

1) entro in NNF2DNNF(nnf)
atoms [b, {a]
2) entro in common_subs(form)
3) entro in common(sublist, group)
list_atoms_sets [{{a}]
list_atoms {{a}
sublist [~b]
3) entro in common(sublist, group)
all_groups [[{a], [~b]]
1) entro in NNF2DNNF(nnf)
atoms [{a]

1) entro in NNF2DNNF(nnf)
atoms [b]

entro in DNNF2dDNNF(dnnf)
> /tmp/ipython-input-2729529069.py(87)DNNF2dDNNF()
     85     print("entro in DNNF2dDNNF(dnnf)")
     86     pdb.set_trace()
---> 87     atoms = list(filter(lambda x: x not in [True, False], dnnf.atoms()))
     88 
     89     print("atoms DNNF2dDNNF", atoms)

ipdb> n
> /tmp/ipython-input-2729529069.py(89)DNNF2dDNNF()
     87     atoms = list(filter(lambda x: x not in [True, False], dnnf.atoms()))
     88 
--->

NameError: name 'deterministic_subs' is not defined

> /tmp/ipython-input-2729529069.py(91)DNNF2dDNNF()
     89     print("atoms DNNF2dDNNF", atoms)
     90     if dnnf.func == And:
---> 91         return reduce(lambda x, y: x & y, deterministic_subs).subs({True | False: True, False | True: True, True & False: False, False & True: False})
     92     result = reduce(lambda x, y: x | y, deterministic_subs).subs({True | False: True, False | True: True, True & False: False, False & True: False})
     93     for subexp1, subexp2 in combinations(deterministic_subs, 2):

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
